Author: Nicholas Steele
Snow Survey Hydrologist
USDA - Natural Resources Conservation Service
1201 NE Lloyd Blvd, Suite #900
Portland, OR  97232
Email: nick.steele@usda.gov
Cell: 503-819-5880


# Imports

module 'xmltodict' is installed


# Dash Web App

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/My Drive/Projects/Work/SNOTELRegressionTool

/content/gdrive/My Drive/Projects/Work/SNOTELRegressionTool


In [1]:
Stations = pd.read_excel('SNOTELStationNames&Triplets.xlsx')
Stations

NameError: name 'pd' is not defined

In [ ]:
stations = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

# stations_complete = dict(zip(stations, triplets))

# labels = {'label': stations}
# values = {'value':triplets}

In [ ]:
options = []
for i in range(len(Stations.index)):
  options.append({'label': stations[i], 'value': triplets[i]})

In [ ]:
options

[{'label': 'Adin Mtn', 'value': '301:CA:SNTL'},
 {'label': 'Cedar Pass', 'value': '391:CA:SNTL'},
 {'label': 'Crowder Flat', 'value': '977:CA:SNTL'},
 {'label': 'Dismal Swamp', 'value': '446:CA:SNTL'},
 {'label': 'State Line', 'value': '1258:CA:SNTL'},
 {'label': 'Aneroid Lake #2', 'value': '302:OR:SNTL'},
 {'label': 'Annie Springs', 'value': '1000:OR:SNTL'},
 {'label': 'Arbuckle Mtn', 'value': '304:OR:SNTL'},
 {'label': 'Bear Grass', 'value': '1166:OR:SNTL'},
 {'label': 'Beaver Reservoir', 'value': '331:OR:SNTL'},
 {'label': 'Big Red Mountain', 'value': '341:OR:SNTL'},
 {'label': 'Bigelow Camp', 'value': '343:OR:SNTL'},
 {'label': 'Billie Creek Divide', 'value': '344:OR:SNTL'},
 {'label': 'Blazed Alder', 'value': '351:OR:SNTL'},
 {'label': 'Blue Mountain Spring', 'value': '357:OR:SNTL'},
 {'label': 'Bourne', 'value': '361:OR:SNTL'},
 {'label': 'Bowman Springs', 'value': '362:OR:SNTL'},
 {'label': 'Cascade Summit', 'value': '388:OR:SNTL'},
 {'label': 'Chemult Alternate', 'value': '395:

In [ ]:
# import itertools

# def zip_with_scalar(l, o):
#     return zip(l, itertools.repeat(o))

In [ ]:
# key = 'label'
# labels = list(map(lambda i: (key, i), Sites))
# labels

In [18]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
from datetime import date
import RegressionFun

In [3]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is B067-5610

 Directory of C:\Users\nicks\Projects\SNOTELRegressionTool

09/29/2021  11:29 AM    <DIR>          .
09/29/2021  11:29 AM    <DIR>          ..
09/29/2021  10:15 AM    <DIR>          .ipynb_checkpoints
09/29/2021  10:14 AM            15,508 RegressionFun.ipynb
09/29/2021  10:14 AM            11,876 RegressionFun.py
09/28/2021  03:19 PM         2,618,800 RegressionTool_v2_0.ipynb
09/29/2021  11:29 AM            32,235 RegressionTool_v2_0_APP.ipynb
09/28/2021  03:50 PM            14,198 SNOTELStationNames_Triplets.xlsx
09/29/2021  10:15 AM    <DIR>          __pycache__
               5 File(s)      2,692,617 bytes
               4 Dir(s)   6,803,058,688 bytes free


In [4]:
Stations = pd.read_excel('SNOTELStationNames_Triplets.xlsx')

In [28]:

#----------------------------------------------------------------------

# variables to be used in different places within app

heading={'color':'black', 'font-size':50,'text-Align':'center','font-weight': 'bold'}
subheading={'color':'black', 'font-weight': 'bold', 'text-Align':'left'}

Stations = pd.read_excel('SNOTELStationNames_Triplets.xlsx')
station_names = Stations.loc[:,'Station Name'].tolist()
triplets = Stations.loc[:, 'Station Triplet'].tolist()

options = []
for i in range(len(Stations.index)):
  options.append({'label': station_names[i], 'value': triplets[i]})
           


#----------------------------------------------------------------------

# Initialize the app class

app = dash.Dash(__name__)
#https://stackoverflow.com/questions/50844844/python-dash-custom-css
#----------------------------------------------------------------------

# app layout

app.layout = html.Div(children = [   
    html.Div(children = [
        html.Br(),
        html.Label(['Select the station to be used as the response:'], style=subheading),
        dcc.Dropdown(id = 'response-station-picker', 
                     options = options, 
                     multi=False,
                     value = 'response'
#                      style={'width': '75%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 'border-color':'black','padding-top': '50px'}                   
        )]),
        
        
        html.Br(),
        html.Label(['Select stations to be used as predictors:'], style=subheading),
        dcc.Dropdown(id = 'predictor-station-picker', 
                     options = options, 
                     multi=True,
                     value = 'predictors'
#                      style={'width': '75%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 'border-color':'black','padding-top': '50px'},
        ),
                   
        html.Br(),
        html.Label(['Select parameter:'], style=subheading),
        dcc.Dropdown(id = 'parameter-picker', 
                     options = 
                     [{'label': 'WTEQ', 'value': 'WTEQ'},
                      {'label': 'PREC', 'value': 'PREC'},
                      {'label': 'SNWD', 'value': 'SNWD'},
                      {'label': 'TAVG', 'value': 'TAVG'},
                      {'label': 'TOBS', 'value': 'TOBS'},
                      {'label': 'TMAX', 'value': 'TMAX'},
                      {'label': 'TMIN', 'value': 'TMIN'}],
            multi=False,
            value="parameter",
#             style={'width': '28%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 
#                   'border-color':'black','padding-top': '50px'}
        ),
    
        html.Br(),
        html.Label(['Select data range to train the regression model: '], style=subheading),
        html.Br(),
        dcc.DatePickerRange(
            id='date-picker-range',
            start_date_placeholder_text='Select a start date!',
            end_date_placeholder_text='Select an end date!',
#             style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'left','border-bottom': 'solid 3px', 
#                   'border-color':'black','padding-top': '50px'}
        ),


        html.Div(children = [
            dcc.Graph(id='training vs test plot', style={'width': '80%','padding-left':'2px','float':'right'}),
            dcc.Graph(id='regression model plot',style={'width': '80%','padding-left':'2px','float':'right'}),
        html.Br(),
        html.Button(id='submit-button-state',
                   children = 'Submit',
#                    style={'fontsize:24'}
        )
        
        ])

        ])








#----------------------------------------------------------------------

# callback for regression plots

# @app.callback(Output('training vs test plot','figure'),
#               [Input('response-station-picker','response'),
#                Input('predictor-station-picker','predictors'),
#                Input('parameter-picker','parameter'),
#                Input('date-picker-range', 'start_date'),
#                Input('date-picker-range', 'end_date')],
#              )

                
@app.callback(Output('training vs test plot','figure'),
              Input('submit-button-state', 'children'),         
               State('response-station-picker','response'),
               State('predictor-station-picker','predictors'),
               State('parameter-picker','parameter'),
               State('date-picker-range', 'start_date'),
               State('date-picker-range', 'end_date'),
 )
             
             
                
                
# @app.callback(Output('regression model plot',component_property='figure'),
#               [Input('response-station-picker',component_property='response'),
#                Input('predictor-station-picker',component_property='predictors'),
#                Input('parameter-picker',component_property='parameter'),
#                Input('date-picker-range', 'start_date'),
#                Input('date-picker-range', 'end_date')
#               ])
             
def plots(response, predictors, parameter, start_date, end_date):
    return {
        
        
        
    }






#----------------------------------------------------------------------------------------------------------
#Run the app

if __name__ == '__main__':
              app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [29/Sep/2021 16:03:01] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [29/Sep/2021 16:03:01] "POST /_dash-update-component HTTP/1.1" 500 -
